[View in Colaboratory](https://colab.research.google.com/github/duncan2708/opencv_extra/blob/2.4/Copy_of_CIFAR_100_Colab.ipynb)

# Using Colab for Image Classification with CIFAR-100

In [0]:
import numpy as np
np.__version__

In [0]:
import tensorflow as tf
tf.__version__

In [0]:
import keras
keras.__version__

Cool. Both Tensorflow and Keras are already available with Colab. And, so are all these!

In [0]:
import sklearn
import cv2
import matplotlib.pyplot as plt

In [0]:
whos

In [0]:
from keras import backend as K
K.set_image_dim_ordering('tf')  

Let's try out hand at some bigger data than MNIST. 

**CIFAR-100** is a labeled subset of the 80 million Tiny Images dataset, and it has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are also grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).

<img src="https://thkimorgblog.files.wordpress.com/2016/03/e18489e185b3e1848fe185b3e18485e185b5e186abe18489e185a3e186ba-2016-03-12-e1848be185a9e1848ce185a5e186ab-2-01-32.png?w=764">

# Getting CIFAR100 from Keras

In [0]:
from keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

Let's examine the data...

In [0]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [0]:
classidx = 0
same_cat = x_train[[np.where(y_train==classidx)][0][0]]
plt.imshow(same_cat[105, :, :, :])
plt.show()

In case you are wondering what are the class labels for classes 0 to 99, here are they...

In [0]:
CIFAR100_LABELS_LIST = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 
    'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 
    'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 
    'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 
    'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
    'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
    'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    'worm'
]

In [0]:
from keras.utils import np_utils
num_classes = 100
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

# Load and download other necessary stuff

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras import optimizers

In [0]:
model = applications.VGG16(include_top=False, weights='imagenet')

# Put data through pre-trained model

In [0]:
model.summary()

In [0]:
bottleneck_features_train = model.predict(x_train, verbose=1 )
np.save(open('vgg16_bottleneck_feats_train.npy', 'wb'), bottleneck_features_train)

In [0]:
bottleneck_features_test = model.predict(x_test, verbose=1 )
np.save(open('vgg16_bottleneck_feats_test.npy', 'wb'), bottleneck_features_test)

In [0]:
!ls

# Train the model

In [0]:
train_data = np.load(open('vgg16_bottleneck_feats_train.npy', 'rb'))

In [0]:
print(train_data.shape)
train_data = train_data[:,0,:,:]
print(train_data.shape)

In [0]:
clf_model = Sequential()
clf_model.add(Flatten(input_shape=(train_data.shape[1:])))
clf_model.add(Dense(256, activation='relu'))
clf_model.add(Dropout(0.5))
clf_model.add(Dense(100, activation='softmax'))

In [0]:
epochs = 50
batch_size = 16

In [0]:
clf_model.summary()


In [0]:
clf_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
tf.test.gpu_device_name()

In [0]:
test_data = np.load(open('vgg16_bottleneck_feats_test.npy', 'rb'))
test_data = test_data[:,0,:,:]

history = clf_model.fit(train_data, y_train, epochs=epochs, batch_size=batch_size, validation_data=(test_data, y_test))

Did your validation accuracy cross 13% ?

In [0]:
clf_model.save('vgg_cifar100_fc_model.h5')

In [0]:
!ls

# Looking into the log

In [0]:
# list all data in history
print(history.history.keys())

In [0]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

From your observation, what should be done next? 

# Fine-tuning the model

We should start by building the "top" end of the CNN. Let's consider the same classifier model we used previously.

In [0]:
clf_model.summary()

In [0]:
# now we should load the conv side from VGG16
#from keras.engine.topology import Input
input_tensor = Input(shape=(32,32,3))
full_model = applications.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

In [0]:
# add the classifier model after the VGG16 convolutional base
clf_model = Sequential()
clf_model.add(Flatten(input_shape=(train_data.shape[1:])))
clf_model.add(Dense(256, activation='relu'))
clf_model.add(Dropout(0.5))
clf_model.add(Dense(100, activation='softmax'))

In [0]:
# note that it is necessary to start with a fully-trained
# classifier, so load the weights trained earlier back
clf_model.load_weights('vgg_cifar100_fc_model.h5')

In [0]:
# add the classifier model on top of the convolutional base
model = Sequential()
model.add(full_model)
model.add(clf_model)
model.summary()

In [0]:
model.get_layer('vgg16').get_layer('block1_conv2')

In [0]:
model.layers[0]

# Fine-tuning the combined model

In [0]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[0].layers[:25]:
    layer.trainable = False

In [0]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='categorical_crossentropy',optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [0]:
history2 = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test))

Show the accuracy and loss plots to further analyze the outcome of your training and validation.